In [1]:
import pickle
import pandas as pd
import os           
import numpy as np
import re
import itertools
import plotly.express as px
#import brightway2 as bw 
import random 
import string

In [2]:
import sys
sys.path.append('../../Module')  #two levels up & then down to Module folder
#from recipe_mod import *
from recipe_mod_resIC import *

<h2> raw RIVM and BW2 only include 5 EFs, same CFs compared to SP/oLCA </h2>

Crude oil; Natural gas; Hard coal; Brown coal; Peat 
<h3> pairwise_comp only for SP vs. oLCA </h3>

In [3]:
lcia_name = "Fossil resource scarcity"
SP_name = "Fossil resource scarcity"
rivm_name = "Fossil resource scarcity"
olca_filepathname = "../../Data_rawCFs/olca_recipe_fossil_saved.dat"
bw2_filepathname = "../../Data_rawCFs/BW2_recipe_Fossil resource scarcity.dat"

# used in final calculation to create dict names
pairwise_name = ["SPvsRIVM", "SPvsBW2", "SPvsopenLCA", "openLCAvsBW2", "openLCAvsRIVM", "BW2vsRIVM"]

### read in SP LCIA

In [4]:
SP_recipe = pd.read_excel("../../Data_rawCFs/SP_ReCiPe.xlsx", sheet_name=SP_name, 
                                  header=[0], converters={'CAS':str})

print(len(SP_recipe))
SP_recipe.fillna(''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(9)), inplace=True)
#to assign a random string_num to CAS with NA, so that they're not the "SAME" CAS when compared to other sources:
#print ([x for x in SP_recipe["CAS"] if '-' not in x ] [0]) 
SP_recipe.columns

21


Index(['Compartment', 'Subcompartment', 'Substance', 'CAS', 'Factor', 'Unit'], dtype='object')

In [5]:
SP_recipe = SP_recipe.rename(columns={"Compartment": "category", "Subcompartment": "subcategory", 
                               "Substance": "flow", "Factor":"value", "CAS": "cas_number" })
SP_recipe = SP_recipe.rename(str.lower, axis='columns')

In [6]:
SP_recipe['flow'] = SP_recipe["flow"].str.lower()
SP_recipe.head()

,category,subcategory,flow,cas_number,value,unit
0,Raw,(unspecified),"coal, 18 mj per kg",FZMZ1E0ML,0.416,kg oil eq / kg
1,Raw,(unspecified),"coal, 26.4 mj per kg",FZMZ1E0ML,0.611,kg oil eq / kg
2,Raw,(unspecified),"coal, 29.3 mj per kg",FZMZ1E0ML,0.678,kg oil eq / kg
3,Raw,(unspecified),"coal, brown",FZMZ1E0ML,0.220,kg oil eq / kg
4,Raw,(unspecified),"coal, hard",FZMZ1E0ML,0.420,kg oil eq / kg


### read in openlca LCIA, delete flow name suffix ", in ground" for pairwise comp

In [7]:
olca_recipe = unpickle_data(olca_filepathname)
olca_recipe.fillna(''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(9)), inplace=True)
#print ([x for x in olca_recipe["CAS_NUMBER"] if '-' not in x ] [0]) 
print(len(olca_recipe))
olca_recipe = olca_recipe.rename(str.lower, axis='columns')

55


In [8]:
olca_recipe['flow'] = olca_recipe["flow"].str.lower()
olca_recipe['flow'] = olca_recipe['flow'].str.split(", in ground").str[0]
olca_recipe.head()

,impact_method,impact_category_uuid,impact_category,reference_unit,flow_uuid,flow,cas_number,subcategory,category,unit,value
0,ReCiPe 2016 Midpoint (H),80e0b1a4-16b9-3aeb-9136-e544aca7a1e3,Fossil resource scarcity,kg oil eq,c5035ce2-5ee5-431f-a287-4b25da42be74,peat,QEIUO81YT,biotic,Resource,kg,0.2200
1,ReCiPe 2016 Midpoint (H),80e0b1a4-16b9-3aeb-9136-e544aca7a1e3,Fossil resource scarcity,kg oil eq,384e875d-2237-4d74-8f62-6b04173f656b,peat,QEIUO81YT,in ground,Resource,kg,0.2200
2,ReCiPe 2016 Midpoint (H),80e0b1a4-16b9-3aeb-9136-e544aca7a1e3,Fossil resource scarcity,kg oil eq,7ae371b0-8532-11e0-9d78-0800200c9a66,"coal, hard, 30.7 mj per kg",QEIUO81YT,in ground,Resource,kg,0.7100
3,ReCiPe 2016 Midpoint (H),80e0b1a4-16b9-3aeb-9136-e544aca7a1e3,Fossil resource scarcity,kg oil eq,5df3c6a9-a888-4b5d-9a94-f3f610d38756,"gas, natural, feedstock, 35 mj per m3",008006-14-2,in ground,Resource,m3,0.8520
4,ReCiPe 2016 Midpoint (H),80e0b1a4-16b9-3aeb-9136-e544aca7a1e3,Fossil resource scarcity,kg oil eq,65b57348-143d-3f95-9229-9c6999296d04,"energy, from oil",QEIUO81YT,in ground,Resource,MJ,0.0218


### read in BW2 LCIA, delete flow name suffix ", in ground" for pairwise comp

In [9]:
bw_recipe = unpickle_data(bw2_filepathname)
bw_recipe.fillna(''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(9)), inplace=True)
#print ([x for x in olca_recipe["CAS_NUMBER"] if '-' not in x ] [0]) 
print(len(bw_recipe))
bw_recipe.columns  #already named in the right format and lower.str : flow, cas_number

5


Index(['cas_number', 'flow', 'flow_uuid', 'type', 'category', 'subcategory',
       'unit', 'value'],
      dtype='object')

In [10]:
bw_recipe['flow'] = bw_recipe["flow"].str.lower()
bw_recipe['flow'] = bw_recipe['flow'].str.split(", in ground").str[0]
bw_recipe

,cas_number,flow,flow_uuid,type,category,subcategory,unit,value
0,4AU99FL5H,"oil, crude",88d06db9-59a1-4719-9174-afeb1fa4026a,natural resource,natural resource,in ground,kilogram,1.00
1,008006-14-2,"gas, natural",7c337428-fb1b-45c7-bbb2-2ee4d29e17ba,natural resource,natural resource,in ground,cubic meter,0.84
2,4AU99FL5H,"coal, hard, unspecified",b6d0042d-0ef8-49ed-9162-a07ff1ccf750,natural resource,natural resource,in ground,kilogram,0.42
3,4AU99FL5H,"coal, brown",024c9722-1e88-412b-8c4b-10c532be8dca,natural resource,natural resource,in ground,kilogram,0.22
4,4AU99FL5H,peat,c5035ce2-5ee5-431f-a287-4b25da42be74,natural resource,natural resource,biotic,kilogram,0.22


### read in raw RIVM_ReCiPe, change flow name to be consistent with others

In [11]:
rivm_recipe = pd.read_excel("../../Data_rawCFs/ReCiPe2016_CFs_v1.1_20180117.xlsx", sheet_name=rivm_name, 
                                  header=[2], converters={'CAS':str})

In [12]:
rivm_recipe['flow'] = ["oil, crude", "gas, natural", "coal, hard, unspecified", "coal, brown", "peat"]

In [13]:
rivm_recipe = rivm_recipe.rename(columns={ "Hierarchist":"value"})
rivm_recipe = rivm_recipe.rename(str.lower, axis='columns')
rivm_recipe

,fossil resource,unit,individualist,value,egalitarian,flow
0,Crude oil,oil-eq/kg,1.00,1.00,1.00,"oil, crude"
1,Natural gas,oil-eq/Nm3,0.84,0.84,0.84,"gas, natural"
2,Hard coal,oil-eq/kg,0.42,0.42,0.42,"coal, hard, unspecified"
3,Brown coal,oil-eq/kg,0.22,0.22,0.22,"coal, brown"
4,Peat,oil-eq/kg,0.22,0.22,0.22,peat


#### compare two methods(A, B)_total unique flows/CAS (regardless of compartment for Resource types of impacts), return: 
    i. an output list incl. below info: 
        (total number of unique flow/CAS in A; "number of flows/CAS" included in A but not in B)
        (total number of unique flow/CAS in B; "number of flows/CAS" included in B but not in A) 
        
    ii. 
        extracting common_unique flows to be used in corr. matrix

### 3. final calculation, pair-wise comparison, corr. matrix

In [14]:
# create 6 empty dict: dict_SPvsRIVM; dict_SPvsBW2; dict_SPvsopenLCA; dict_openLCAvsBW2; dict_openLCAvsRIVM; dict_BW2vsRIVM
# each dict with four keys and null values (to be assigned later)
for name in pairwise_name:
    #print(name)
    keys = [name+"_commonEF_sumtable", name+"_commonEF_list", name + "_result_diffEFs", name + "_result_corr" ]
    globals()['dict_%s' % name] = dict.fromkeys(keys, None)

In [15]:
k_list = list(dict_SPvsRIVM)
dict_SPvsRIVM[k_list[0]], dict_SPvsRIVM[k_list[1]] = res_compare_included_EF(SP_recipe, "flow", rivm_recipe,  "flow", to_print = "yes" )
if len(dict_SPvsRIVM[k_list[1]] ) == 0: 
    dict_SPvsRIVM[k_list[2]] = pd.DataFrame ({'SP_value': np.NaN , 'RIVM_value': np.NaN }, index=[0])
else:
    dict_SPvsRIVM[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP_recipe, "flow", "value", rivm_recipe, "flow", "value", dict_SPvsRIVM[k_list[1]])[0] , 
         'RIVM_value': final_EF_combined(SP_recipe, "flow", "value", rivm_recipe, "flow", "value", dict_SPvsRIVM[k_list[1]])[1] }, 
        index = dict_SPvsRIVM[k_list[1]]
    )

EF in A but not in B: ['energy, from oil', 'coal, hard', 'oil, crude, 42 mj per kg', 'coal, 26.4 mj per kg', 'oil, crude, 42.7 mj per kg', 'coal, 29.3 mj per kg', 'oil, crude, 41 mj per kg', 'energy, from gas, natural', 'gas, natural, 36.6 mj per m3', 'gas, natural, feedstock, 35 mj per m3', 'coal, 18 mj per kg', 'oil, crude, feedstock, 42 mj per kg', 'oil, crude, 42.6 mj per kg', 'gas, natural/m3', 'coal, hard, 30.7 mj per kg', 'gas, natural, 35 mj per m3', 'energy, from coal', 'oil, crude, feedstock, 41 mj per kg']
EF in B but not in A: ['coal, hard, unspecified', 'gas, natural']
N of common EFs to be used is: 3


In [17]:
k_list = list(dict_SPvsBW2)
dict_SPvsBW2[k_list[0]],dict_SPvsBW2[k_list[1]] = res_compare_included_EF(SP_recipe, "flow", bw_recipe,  "flow", to_print = "yes" )
if len(dict_SPvsBW2[k_list[1]] ) == 0: 
    dict_SPvsBW2[k_list[2]] = pd.DataFrame ({'SP_value': np.NaN , 'BW2_value': np.NaN }, index=[0])
else:
    dict_SPvsBW2[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP_recipe, "flow", "value", bw_recipe, "flow", "value", dict_SPvsBW2[k_list[1]])[0] , 
         'BW2_value': final_EF_combined(SP_recipe, "flow", "value", bw_recipe, "flow", "value", dict_SPvsBW2[k_list[1]])[1] }, 
        index = dict_SPvsBW2[k_list[1]]
    )

EF in A but not in B: ['energy, from oil', 'coal, hard', 'oil, crude, 42 mj per kg', 'coal, 26.4 mj per kg', 'oil, crude, 42.7 mj per kg', 'coal, 29.3 mj per kg', 'oil, crude, 41 mj per kg', 'energy, from gas, natural', 'gas, natural, 36.6 mj per m3', 'gas, natural, feedstock, 35 mj per m3', 'coal, 18 mj per kg', 'oil, crude, feedstock, 42 mj per kg', 'oil, crude, 42.6 mj per kg', 'gas, natural/m3', 'coal, hard, 30.7 mj per kg', 'gas, natural, 35 mj per m3', 'energy, from coal', 'oil, crude, feedstock, 41 mj per kg']
EF in B but not in A: ['coal, hard, unspecified', 'gas, natural']
N of common EFs to be used is: 3


In [18]:
k_list = list(dict_SPvsopenLCA)
dict_SPvsopenLCA[k_list[0]],dict_SPvsopenLCA[k_list[1]] = res_compare_included_EF(SP_recipe, "flow", olca_recipe,  "flow", to_print = "yes" )
if len(dict_SPvsopenLCA[k_list[1]] ) == 0: 
    dict_SPvsopenLCA[k_list[2]] = pd.DataFrame ({'SP_value': np.NaN , 'olca_value': np.NaN }, index=[0])
else:
    dict_SPvsopenLCA[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP_recipe, "flow", "value", olca_recipe, "flow", "value", dict_SPvsopenLCA[k_list[1]])[0] , 
         'olca_value': final_EF_combined(SP_recipe, "flow", "value", olca_recipe, "flow", "value", dict_SPvsopenLCA[k_list[1]])[1] },
        index = dict_SPvsopenLCA[k_list[1]]
    )

EF in A but not in B: []
EF in B but not in A: ['gas, natural', 'coal, hard, unspecified']
N of common EFs to be used is: 21


In [19]:
k_list = list(dict_openLCAvsBW2)
dict_openLCAvsBW2[k_list[0]],dict_openLCAvsBW2[k_list[1]] = res_compare_included_EF(olca_recipe, "flow_uuid", bw_recipe, "flow_uuid", to_print = "no" )

if len(dict_openLCAvsBW2[k_list[1]] ) == 0: 
    dict_openLCAvsBW2[k_list[2]] = pd.DataFrame ({'olca_value': np.NaN , 'BW2_value': np.NaN }, index=[0])
else:
    dict_openLCAvsBW2[k_list[2]] = pd.DataFrame (
        {'olca_value': final_EF_combined(olca_recipe, "flow_uuid", "value", bw_recipe, "flow_uuid", "value", 
                                       dict_openLCAvsBW2[k_list[1]])[0] , 
         'BW2_value': final_EF_combined(olca_recipe, "flow_uuid", "value", bw_recipe, "flow_uuid", "value",
                                         dict_openLCAvsBW2[k_list[1]])[1] },
        index =  dict_openLCAvsBW2[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 5


In [20]:
k_list = list(dict_openLCAvsRIVM)
dict_openLCAvsRIVM[k_list[0]],dict_openLCAvsRIVM[k_list[1]] = res_compare_included_EF(olca_recipe, "flow", rivm_recipe, "flow", to_print = "no" )

if len(dict_openLCAvsRIVM[k_list[1]] ) == 0: 
    dict_openLCAvsRIVM[k_list[2]] = pd.DataFrame ({'olca_value': np.NaN , 'RIVM_value': np.NaN }, index=[0])
else:
    dict_openLCAvsRIVM[k_list[2]] = pd.DataFrame (
        {'olca_value': final_EF_combined(olca_recipe, "flow", "value", rivm_recipe, "flow", "value", 
                                       dict_openLCAvsRIVM[k_list[1]])[0] , 
         'BW2_value': final_EF_combined(olca_recipe, "flow", "value", rivm_recipe, "flow", "value",
                                         dict_openLCAvsRIVM[k_list[1]])[1] },
        index = dict_openLCAvsRIVM[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 5


In [21]:
k_list = list(dict_BW2vsRIVM)
dict_BW2vsRIVM[k_list[0]],dict_BW2vsRIVM[k_list[1]] = res_compare_included_EF(bw_recipe, "flow", rivm_recipe, "flow", to_print = "yes" )
if len(dict_BW2vsRIVM[k_list[1]] ) == 0: 
    dict_BW2vsRIVM[k_list[2]] = pd.DataFrame ({'BW2_value': np.NaN , 'RIVM_value': np.NaN }, index=[0])
else:
    dict_BW2vsRIVM[k_list[2]] = pd.DataFrame (
        {'BW2_value': final_EF_combined(bw_recipe, "flow", "value", rivm_recipe, "flow", "value", 
                                       dict_BW2vsRIVM[k_list[1]])[0] , 
         'RIVM_value': final_EF_combined(bw_recipe, "flow", "value", rivm_recipe, "flow", "value",
                                         dict_BW2vsRIVM[k_list[1]])[1] },
        index = dict_BW2vsRIVM[k_list[1]]
    )

EF in A but not in B: []
EF in B but not in A: []
N of common EFs to be used is: 5


In [22]:
for name in pairwise_name:
    new_df = globals()['dict_%s' % name][name + "_result_diffEFs"]
    try:
        new_df_corr = new_df.corr()
    except: 
        error
    globals()['dict_%s' % name][name + "_result_corr"] = new_df_corr

In [23]:
dict_SPvsRIVM['SPvsRIVM_commonEF_sumtable']

,Method_A,Method_B
total_uniq_EF,21,5
EF_notin_another,18,2


In [24]:
dict_SPvsRIVM['SPvsRIVM_commonEF_list']

{'coal, brown', 'oil, crude', 'peat'}

In [25]:
dict_SPvsRIVM['SPvsRIVM_result_diffEFs']

,SP_value,RIVM_value
"coal, brown",0.22,0.22
peat,0.22,0.22
"oil, crude",1.00,1.00


In [26]:
dict_SPvsRIVM['SPvsRIVM_result_corr']

,SP_value,RIVM_value
SP_value,1.0,1.0
RIVM_value,1.0,1.0


In [27]:
# saving each dict_ to a single worksheet, different rows , # saving to sub-folder "results/Resources_IC/"

if not os.path.exists("results/Resources_IC"):
    os.makedirs("results/Resources_IC")

filename = lcia_name + "_pairwise_comp_result.xlsx"
writer = pd.ExcelWriter("results/Resources_IC/" + filename, engine='xlsxwriter')

for name in pairwise_name:
    sum_table = globals()['dict_%s' % name][name + "_commonEF_sumtable"]
    new_df = globals()['dict_%s' % name][name + "_result_diffEFs"]
    new_df_corr = globals()['dict_%s' % name][name + "_result_corr"]

    sum_table.to_excel(writer,sheet_name=name,startrow=0 , startcol=0)   # write first  (3rows)
    new_df_corr.to_excel(writer,sheet_name=name,startrow=5 , startcol=0) # write next   (3rows)
    new_df.to_excel(writer,sheet_name=name,startrow=10 , startcol=0)     # finally, write the resulting EFs 
     
writer.save()